In [202]:
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip
from collections import Counter
import pprint

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
df = pd.read_csv('DIPS_Data.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df = df.dropna(how='all')

In [203]:
#assigns values to each row in the column and removes blank rows
data = df.copy()
data = data.fillna({'case_name':'none'})
data['Case'] = 'xxxx'
value = 'Fees:'
case = 'xxxx'
for row in data.itertuples():
    if row[1] not in ['none', 'Fees:', 'Agents:','Lender Groups:', 'Roll ups:', 'Lenders:']:
        case = row[1]
        data.at[row[0], 'Case'] = case
        continue
    if row[1] == 'Fees:':
        value = 'Fees:'
        continue
    if row[1] == 'Agents:':
        value = 'Agents:'
        continue
    if row[1] == 'Lender Groups:':
        value = 'Lender Groups:'
        continue
    if row[1] == 'Roll ups:':
        value = 'Roll ups:'
        continue
    if row[1] == 'Lenders:':
        value = 'Lenders:'
        continue
    data.at[row[0], 'Case'] = case
    data.at[row[0], 'case_name'] = value
data = data[~data.Case.isin(['xxxx'])]
data = data.reset_index(drop=True)
#data = data.set_index('Case')

In [156]:
data.head(20)

,case_name,case_status,petition_date,confirmation_date,dismissal_date,closing_date,effective_date,unnamed:_7,publishing_name,currency,loan_amount,maturity,facility_type,facility_sub_type,facility_other_type,unnamed:_15,fixed/floating,pik,amount,floating_type,floating_amount,unnamed:_21,investigation_budget,ucc_sublimit,carve_out,professional_breakdown,deadline,sublimit_amount,unnamed:_28,priotity/prime_everything,notes,collateral_all_assets,notes.1,adequate_protection,min_liquidity_covenant,variance,professional_line_item,professional_line_item_period,Case
0,In re Fairway Group Holdings Corp.,confirmed,5/2/2016,6/8/2016,NaN,NaN,7/3/2016,NaN,DIP Term Loan,USD,55000000.0,3.0,Term Loan,NaN,NaN,NaN,Floating,NaN,NaN,LIBOR,8.00,NaN,250000.0,Yes,1000000.0,NaN,30.0,30.0,NaN,Y,NaN,Y,NaN,"Liens,FeesExpenses",NaN,15.0,542000000,13,In re Fairway Group Holdings Corp.
1,Fees:,Unused Commitment Fee,NaN,0.375,%,Monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
2,Fees:,Other Fee,Duration Fee,0.25,%,One-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
3,Fees:,Commitment Fee,NaN,2,%,One-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
4,Fees:,Arrangement/Origination Fee,NaN,0.5,%,One-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
5,Agents:,Credit Suisse,Admin Agent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
6,Agents:,Credit Suisse,Collateral Agent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
7,Lender Groups:,Consenting Lender Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In re Fairway Group Holdings Corp.
8,"In re Arch Coal, Inc.",confirmed,1/11/2016,9/15/2016,NaN,10/5/2016,10/5/2016,NaN,DIP Term Loan,USD,275000000.0,12.0,Term Loan,NaN,NaN,NaN,Floating,NaN,NaN,LIBOR,9.00,NaN,250000.0,NaN,10000000.0,NaN,64.0,NaN,NaN,Y,Additional up to $75mm carve-out in favor of c...,N,Collateral package excludes Receivables Rights,"Liens,Interest,FeesExpenses,FinancialInformation",Yes,NaN,NaN,NaN,"In re Arch Coal, Inc."
9,Fees:,Unused Commitment Fee,NaN,5,%,Annually,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In re Arch Coal, Inc."


In [176]:
original_admin_list = list(data[data.petition_date.isin(['Admin Agent', 'Indenture Trustee', 'Lender'])]['Case'])
len(original_admin_list)

220

In [204]:
starting_list = list(data['case_name'])
clean_list = [x for x in starting_list if str(x) != 'nan']
unique_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
all_cases = [x for x in clean_list if str(x) not in ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']]
total_cases = len(all_cases)
#counts the total number of cases present in the dataframe
#observation_count = Counter(clean_list) #dictionary of count of unique deals, Fees, Agents ect.

#combs through the created list and returns the case names that have missing data i.e No Agents, No Lenders ect.
master_list = ['Agents:', 'Fees:', 'Lender Groups:', 'Lenders:', 'Roll ups:']
case_dict = {}
case_index = []
agent_no_lenders = []
no_data = []
lenders_no_agent = []
one_lender_no_agent = []
no_agent = []

for i in range(0, len(clean_list)):
    if clean_list[i] not in master_list:
        fee_num = 0
        agent_num = 0
        name = clean_list[i]
        case_index.append(i)
        dup_case = name in case_dict
        if not dup_case:
            case_dict[name] = {}
            case_dict[name]['Case_Count'] = 1
            dip_num = name + ': ' + str(1)
        if dup_case:
            case_dict[name]['Case_Count'] = case_dict[name]['Case_Count'] + 1
            dip_num = name + ': ' + str(case_dict[name]['Case_Count'])
        case_dict[name][dip_num] = {}
        case_dict[name][dip_num]['Data'] = {}
        case_dict[name][dip_num]['Data']['Publishing_Name'] = data.loc[i]['publishing_name']
        case_dict[name][dip_num]['Data']['Currency'] = data.loc[i]['currency']
        case_dict[name][dip_num]['Data']['Loan_Amount'] = data.loc[i]['loan_amount']
        case_dict[name][dip_num]['Data']['Maturity'] = data.loc[i]['maturity']
        case_dict[name][dip_num]['Data']['Facility_Type'] = data.loc[i]['facility_type']
        case_dict[name][dip_num]['Data']['Fixed_Float'] = data.loc[i]['fixed/floating']
        case_dict[name][dip_num]['Data']['Floating_Type'] = data.loc[i]['floating_type']
        case_dict[name][dip_num]['Data']['Investigation_Budget'] = data.loc[i]['investigation_budget']
        case_dict[name][dip_num]['Data']['Carve_Out'] = data.loc[i]['carve_out']
    if clean_list[i] == 'Fees:':
        if fee_num == 0:
            case_dict[name][dip_num]['Fees'] = {}
        fee_num = fee_num + 1
        fee_name = 'Fee_'+str(fee_num)
        case_dict[name][dip_num]['Fees'][fee_name] = {}
        case_dict[name][dip_num]['Fees'][fee_name]['Type'] = data.loc[i]['case_status']
        case_dict[name][dip_num]['Fees'][fee_name]['Sub_Type'] = data.loc[i]['petition_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Amount'] = data.loc[i]['confirmation_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Amount_Type'] = data.loc[i]['dismissal_date']
        case_dict[name][dip_num]['Fees'][fee_name]['Duration_Type'] = data.loc[i]['closing_date']
    if clean_list[i] == 'Agents:':
        agent_type = str(data.loc[i]['petition_date'])
        if str(data.loc[i]['petition_date']) in ['Lender', 'Indenture Trustee']:
            agent_type = 'Admin Agent'
        if 'Agents' not in case_dict[name][dip_num]:
            case_dict[name][dip_num]['Agents'] = {}
            case_dict[name][dip_num]['Agents'][agent_type] = []
        if 'Agents' in case_dict[name][dip_num]:
            if agent_type not in case_dict[name][dip_num]['Agents']:
                case_dict[name][dip_num]['Agents'][agent_type] = []
        case_dict[name][dip_num]['Agents'][agent_type].append(str(data.loc[i]['case_status']))
        

In [149]:
case_dict['In re Toys "R" Us, Inc.']

{'Case_Count': 6,
 'In re Toys "R" Us, Inc.: 1': {'Data': {'Publishing_Name': 'DIP Revolver',
   'Currency': 'USD',
   'Loan_Amount': 1850000000.0,
   'Maturity': 16.0,
   'Facility_Type': 'Revolver',
   'Fixed_Float': 'Floating',
   'Floating_Type': 'LIBOR',
   'Investigation_Budget': 500000.0,
   'Carve_Out': 20000000.0},
  'Fees': {'Fee_1': {'Type': 'Other Fee',
    'Sub_Type': 'USD 95.5m in total fees for all DIPs',
    'Amount': nan,
    'Amount_Type': 'USD',
    'Duration_Type': 'One-time'},
   'Fee_2': {'Type': ' Unused Commitment Fee',
    'Sub_Type': nan,
    'Amount': '0.375',
    'Amount_Type': '%',
    'Duration_Type': 'Annually'}},
  'Agents': {'Admin Agent': ['JPMorgan']}},
 'In re Toys "R" Us, Inc.: 2': {'Data': {'Publishing_Name': 'DIP USD 450m Term Loan (FILO)',
   'Currency': 'USD',
   'Loan_Amount': 450000000.0,
   'Maturity': 16.0,
   'Facility_Type': 'Term Loan',
   'Fixed_Float': 'Floating',
   'Floating_Type': 'LIBOR',
   'Investigation_Budget': 50000.0,
   'Carv

In [205]:
print('Case Count: ', total_cases)
dict_count = 0
admin_count = 0
no_admin_agent = []
admin_name = []
all_admin = []
multiple_admin = []
for i in case_dict.keys():
    dict_count += case_dict[i]['Case_Count']
    for j in range(1, case_dict[i]['Case_Count']+1):
        if 'Agents' not in case_dict[i][i+': '+str(j)]:
            continue
        if 'Agents' in case_dict[i][i+': '+str(j)]:
            if 'Admin Agent' in case_dict[i][i+': '+str(j)]['Agents']:
                if len(case_dict[i][i+': '+str(j)]['Agents']['Admin Agent']) > 1:
                    multiple_admin.append(i)
                admin_name.append(case_dict[i][i+': '+str(j)]['Agents']['Admin Agent'][0])
                all_admin.append(i)
                admin_count += 1
            else:
                no_admin_agent.append(i+': '+str(j))
                continue

print('Dictionary Case Count: ', dict_count)
#print('Admin Agent Total: ', admin_agent_total)
print('Dictionary Admin Agent Count: ', admin_count)
multiple_admin

Case Count:  347
Dictionary Case Count:  347
Dictionary Admin Agent Count:  215


['In re Emerald Oil, Inc.',
 'In re Sotera Wireless, Inc.',
 'In re Maurice Sporting Goods, Inc.',
 'In re Garden Fresh Restaurant Intermediate Holding, LLC',
 'In re The Bon-Ton Stores, Inc.']

In [201]:
case_dict['In re Expro Holdings US Inc.']

{'Case_Count': 2,
 'In re Expro Holdings US Inc.: 1': {'Data': {'Publishing_Name': 'DIP Term Loan',
   'Currency': 'USD',
   'Loan_Amount': 125000000.0,
   'Maturity': 6.0,
   'Facility_Type': 'Term Loan',
   'Fixed_Float': 'Floating',
   'Floating_Type': 'LIBOR',
   'Investigation_Budget': nan,
   'Carve_Out': 5000000.0},
  'Agents': {'Admin Agent': ['JPMorgan', 'Angelo, Gordon & Co.']}},
 'In re Expro Holdings US Inc.: 2': {'Data': {'Publishing_Name': 'DIP Bonding Facility',
   'Currency': 'USD',
   'Loan_Amount': 30000000.0,
   'Maturity': nan,
   'Facility_Type': 'Other',
   'Fixed_Float': 'Fixed',
   'Floating_Type': 'LIBOR',
   'Investigation_Budget': nan,
   'Carve_Out': 5000000.0},
  'Fees': {'Fee_1': {'Type': ' Unused Commitment Fee',
    'Sub_Type': nan,
    'Amount': '0.625',
    'Amount_Type': '%',
    'Duration_Type': 'Annually'}}}}

In [180]:
from collections import Counter
print('Number of Original Admin, Lenders, and Indenture Trustees: ', len(original_admin_list))
print('Number of cleaned Admin, Lenders, and Indenture Trustees: ', len(all_admin))
clean_admin = Counter(all_admin)
original_admin = Counter(original_admin_list)

Number of Original Admin, Lenders, and Indenture Trustees:  220
Number of cleaned Admin, Lenders, and Indenture Trustees:  215


In [116]:
added_agents = pd.read_csv('Added_Agents.csv')

In [193]:
added_agents.head(20)

,Case,case_name,case_status,petition_date
0,"In re AF-Southeast, LLC (Allied Fiber)",Agents:,Strome Mezzanine Fund,Admin Agent
1,"In re AOG Entertainment, Inc. (Core Entertainm...",Agents:,"Elvis Blue Moon Holdings, LLC",Admin Agent
2,"In re AstroTurf, LLC",Agents:,"Textile Management Associates, Inc.",Admin Agent
3,In re EMAS CHIYODA Subsea Limited,Agents:,"Subsea 7, S.A.",Admin Agent
4,"In re Filip Technologies, Inc.",Agents:,"AT&T Services, Inc.",Admin Agent
5,"In re FPMC Austin Realty Partners, LP",Agents:,Frost Bank,Admin Agent
6,In re Daily Gazette Company,Agents:,United Bank,Admin Agent
7,In re EMAS CHIYODA Subsea Limited,Agents:,Chiyoda Corporation,Admin Agent
8,In re Bear Metallurgical Company (Gulf Chemical),Agents:,Comilog Holding,Admin Agent
9,"In re Blue Earth, Inc.",Agents:,Jackson Investment Group,Admin Agent


In [190]:
case_dict['']

{'In re Fairway Group Holdings Corp.': {'Case_Count': 2,
  'In re Fairway Group Holdings Corp.: 1': {'Data': {'Publishing_Name': 'DIP Term Loan',
    'Currency': 'USD',
    'Loan_Amount': 55000000.0,
    'Maturity': 3.0,
    'Facility_Type': 'Term Loan',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 250000.0,
    'Carve_Out': 1000000.0},
   'Fees': {'Fee_1': {'Type': ' Unused Commitment Fee',
     'Sub_Type': nan,
     'Amount': '0.375',
     'Amount_Type': '%',
     'Duration_Type': 'Monthly'},
    'Fee_2': {'Type': 'Other Fee',
     'Sub_Type': 'Duration Fee',
     'Amount': '0.25',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_3': {'Type': 'Commitment Fee',
     'Sub_Type': nan,
     'Amount': '2',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_4': {'Type': 'Arrangement/Origination Fee',
     'Sub_Type': nan,
     'Amount': '0.5',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'}},
   'A

In [198]:
added_dict = {}
for i in range(0, len(added_agents.index)):
    name = str(added_agents.loc[i]['Case'])
    agent = str(added_agents.loc[i]['case_status'])
    agent_type = str(added_agents.loc[i]['petition_date'])
    #print(name)
    if name in added_dict:
        added_dict[name] = added_dict[name] + 1
    if name not in added_dict:
        added_dict[name] = 1
    dip_num = name + ': ' + str(added_dict[name])
    #print(name)
    #print(dip_num)
    if 'Agents' in case_dict[name][dip_num]:
        if agent_type not in case_dict[name][dip_num]['Agents']:
            case_dict[name][dip_num]['Agents'][agent_type] = [agent]
        if agent_type in case_dict[name][dip_num]['Agents']:
            case_dict[name][dip_num]['Agents'][agent_type].append(agent)
    if 'Agents' not in case_dict[name][dip_num]:
        case_dict[name][dip_num]['Agents'] = {}
        case_dict[name][dip_num]['Agents'][agent_type] = [agent]

In [208]:
case_index = []
dip_number_index = []
data_pull = []
#key_list = list(case_dict.keys()
for i in list(case_dict.keys()):
    for j in range(1, case_dict[i]['Case_Count']+1):
        if 'Agents' in case_dict[i][i + ': ' + str(j)]:
            if 'Admin Agent' in case_dict[i][i + ': ' + str(j)]['Agents']:
                case_index.append(i)
                dip_number_index.append(i + ': ' + str(j))
                data_pull.append([case_dict[i][i + ': ' + str(j)]['Data']['Loan_Amount'], case_dict[i][i + ': ' + str(j)]['Agents']['Admin Agent'][0]])
                #loan_amount_column.append(case_dict[i]['Dip_Number_'+str(j)]['Data']['Loan_Amount'])
                #admin_agent_column.append(case_dict[i]['Dip_Number_'+str(j)]['Agents']['Admin Agent'][0])
                
                #puller_list.append([case_dict[i]['Dip_Number_'+str(j)]['Data'], case_dict[i]['Dip_Number_'+str(j)]['Agents']['Admin Agent']])

In [212]:
pd.DataFrame(data=data_pull, index = [case_index, dip_number_index], columns=['loan_amount', 'admin_agent'])#.to_csv('Agents_Loan_Amount.csv',encoding='utf-8')

loan_amount                                admin_agent
In re Fairway Group Holdings Corp.                 In re Fairway Group Holdings Corp.: 1               5.500000e+07                              Credit Suisse
                                                   In re Fairway Group Holdings Corp.: 2               3.061194e+07                              Credit Suisse
In re Arch Coal, Inc.                              In re Arch Coal, Inc.: 1                            2.750000e+08                           Wilmington Trust
In re Aspect Software Parent, Inc.                 In re Aspect Software Parent, Inc.: 1               3.000000e+07                           Wilmington Trust
In re Peabody Energy Corporation                   In re Peabody Energy Corporation: 1                 5.000000e+08                                   CitiBank
                                                   In re Peabody Energy Corporation: 2                 2.000000e+08                                   CitiBank
                                                   In re Peabody Energy Corporation: 3                 1.000000e+08                                   CitiBank
In re CJ Holding Co. (C&amp;J Energy)              In re CJ Holding Co. (C&amp;J Energy): 1            1.000000e+08           Cortland Capital Market Services
In re American Gilsonite Company                   In re American Gilsonite Company: 1                 3.000000e+07                           Wilmington Trust
In re Phoenix Brands LLC                           In re Phoenix Brands LLC: 1                         2.570431e+07                    Madison Capital Funding
In re Penn Virginia Corporation                    In re Penn Virginia Corporation: 1                  2.500000e+07                           Wells Fargo Bank
In re Erickson Incorporated                        In re Erickson Incorporated: 1                      6.667000e+07            Wilmington Savings Fund Society
                                                   In re Erickson Incorporated: 2                      1.160000e+08                           Wells Fargo Bank
In re Gawker Media, LLC                            In re Gawker Media, LLC: 1                          5.000000e+06                Cerberus Capital Management
                                                   In re Gawker Media, LLC: 2                          1.700000e+07                Cerberus Capital Management
In re Emerald Oil, Inc.                            In re Emerald Oil, Inc.: 1                          1.299280e+08                           Wells Fargo Bank
In re Draw Another Circle, LLC                     In re Draw Another Circle, LLC: 1                   9.000000e+07                            Bank of America
In re Global Geophysical Services, LLC (Autoseis)  In re Global Geophysical Services, LLC (Autosei...  2.000000e+06            Wilmington Savings Fund Society
In re Halcon Resources Corporation                 In re Halcon Resources Corporation: 1               6.000000e+08                                   JPMorgan
In re Horsehead Holding Corp.                      In re Horsehead Holding Corp.: 1                    9.000000e+07                          Cantor Fitzgerald
In re Linc USA GP (Linc Energy)                    In re Linc USA GP (Linc Energy): 1                  1.000000e+07                          Cantor Fitzgerald
In re Roadhouse Holding Inc. (Logan's Roadhouse)   In re Roadhouse Holding Inc. (Logan's Roadhouse...  7.500000e+07           Cortland Capital Market Services
In re RCS Capital Corporation                      In re RCS Capital Corporation: 1                    1.000000e+08                                   Barclays
In re Rotary Drilling Tools USA, LLC               In re Rotary Drilling Tools USA, LLC: 1             3.000000e+06                                   PNC Bank
In re Seventy Seven Finance Inc. (Seventy Seven... In re Seventy Seven Finance Inc. (Seventy Seven...  1.000000e+08              

In [97]:
case_dict

{'In re Fairway Group Holdings Corp.': {'Case_Count': 2,
  'Dip_Number_1': {'Data': {'Publishing_Name': 'DIP Term Loan',
    'Currency': 'USD',
    'Loan_Amount': 55000000.0,
    'Maturity': 3.0,
    'Facility_Type': 'Term Loan',
    'Fixed_Float': 'Floating',
    'Floating_Type': 'LIBOR',
    'Investigation_Budget': 250000.0,
    'Carve_Out': 1000000.0},
   'Fees': {'Fee_1': {'Type': ' Unused Commitment Fee',
     'Sub_Type': nan,
     'Amount': '0.375',
     'Amount_Type': '%',
     'Duration_Type': 'Monthly'},
    'Fee_2': {'Type': 'Other Fee',
     'Sub_Type': 'Duration Fee',
     'Amount': '0.25',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_3': {'Type': 'Commitment Fee',
     'Sub_Type': nan,
     'Amount': '2',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'},
    'Fee_4': {'Type': 'Arrangement/Origination Fee',
     'Sub_Type': nan,
     'Amount': '0.5',
     'Amount_Type': '%',
     'Duration_Type': 'One-time'}},
   'Agents': {'Admin Agent': [